In [1]:
import torch
from torch_scatter import scatter

In [2]:
x = torch.randn((9, 4))
cluster = torch.cat((torch.zeros(4), torch.ones(5))).long()
print(x)
print(cluster)

tensor([[ 0.1266, -1.2514, -0.4005, -0.5617],
        [ 0.7433,  1.0250, -0.2480,  0.5520],
        [ 0.3471,  0.3888,  0.1874, -0.4121],
        [ 1.0442, -0.9976,  0.3196, -0.9103],
        [-0.6950,  1.4699, -1.3755, -0.9901],
        [ 0.0303, -1.4798,  1.1173,  0.5543],
        [-0.3560, -1.0236, -1.9061,  0.6711],
        [ 0.3916, -0.1797, -0.9383, -0.4893],
        [-1.9653,  0.2235, -0.1763,  2.2131]])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1])


In [5]:
c = scatter(x, cluster, dim=0, reduce='max')
print(c)
print(c.shape)
print(c[cluster])

tensor([[1.0442, 1.0250, 0.3196, 0.5520],
        [0.3916, 1.4699, 1.1173, 2.2131]])
torch.Size([2, 4])
tensor([[1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [1.0442, 1.0250, 0.3196, 0.5520],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131],
        [0.3916, 1.4699, 1.1173, 2.2131]])


In [4]:
def scatter_naive1D(src, index, dim, dim_size, reduce):
    # Assume 1D tensor. 
    assert dim == 0 and src.dim() == 1 and index.dim() == 1, "Only support 1D scatter!"

    def reduce_op(l, op):
        result = 0
        if op == 'min':
            result = torch.min(l)
        elif op == 'max':
            result = torch.max(l)
        elif op == 'sum':
            result = torch.sum(l)
        elif op == 'mean':
            result = torch.mean(l)
        else:
            assert False, 'Reduce op not supported!'

        return result 

    output = torch.zeros(dim_size, dtype=src.dtype, device=src.device)

    index_range = torch.max(index) + 1
    bucket = [[] for _ in range(index_range)]

    for i in range(len(src)):
        bucket[index[i]].append(src[i])

    for i in range(len(bucket)):
        if bucket[i]:
            bucket[i] = reduce_op(torch.as_tensor(bucket[i], dtype=src.dtype, device=src.device), reduce)
        else:
            bucket[i] = 0

    output[:len(bucket)] = torch.as_tensor(bucket, dtype=src.dtype, device=src.device)

    return output

In [9]:
scatter_naive1D(torch.tensor([19,2,3,4,5]), torch.tensor([0,0,1,1,1]), 0, (2,), 'max')

tensor([19,  5])